# Lab 4: PI Control

This lab assignment introduces the use of PID control for the temperature control laboratory. In this assignment you will be 

* testing a simple implementation of PI control
* identifying issues that have to be addressed for a more practical implementation

## Implementation of a simple PI controller

Given a process variable $PV$ and setpoint $SP$, proportional-integral-derivative control determines the value of a manipulated variable MV by the equation

TODO: Make consistent with rest of website. Check sign convention.

\begin{align}
MV & = \bar{MV} + k_p\left(SP - PV\right) + k_i \int_0^t \left(SP-PV)\right)dt
\end{align}

where $k_p$ and $k_i$ are the proportional and integral coefficients, respectively. The value $\bar{MV}$ is a nominal or initial value of the manipulated variable.

The actual implementation of PI control is normally done by computing how much the $MV$ should change at each time step. Defining the error at time $k$ as

\begin{align}
e_k & = SP_k - PV_k
\end{align}

then consecutive values of $MV$ are given by 

\begin{align}
MV_{k-1} & = \bar{MV} + k_p e_{k-1} + k_i \sum_{j=0}^{k-1} e_{j} \\
MV_{k} & = \bar{MV} + k_p e_{k} + k_i \sum_{j=0}^{k} e_{j}
\end{align}

Taking differences gives a practical formula for updating the value of $MV$ in response to measurements

\begin{align}
MV_{k} & = MV_{k-1} + k_p(e_{k} - e_{k-1}) + k_i e_{k}
\end{align}

The following code defines a Python object that implements this algorithm.

In [ ]:
def PI(Kp=1, Ki=0, MV_bar=0):
    MV = MV_bar
    e_prev = 0
    MV_min = 0
    MV_max = 100
    while True:
        t_step, SP, PV, MV = yield MV
        #e = PV - SP # Calculate error
        e = SP - PV
        #MV += -Kp*(e - e_prev) - t_step*Ki*e # Apply PI formula
        MV += Kp*(e - e_prev) + t_step*Ki*e
        MV = max(MV_min, min(MV_max, MV)) # Apply upper and lower bounds
        e_prev = e

## Exercise 1. Tune the PI control for the Temperature Control Lab

The following cell provides an initial implementation of PI control for heater T1.  This is setup for testing with the off-line simulation mode of tclab.  Experiment with the simulation to find appropriate values for $k_p$ and $k_i$. Your design goal is to achieve the setpoint and stay within a zone of +/- 2 degrees as quickly as possible.

In [ ]:
%matplotlib inline
from tclab import setup, clock, Historian, Plotter

In [ ]:
TCLab = setup(connected=False, speedup = 20)

SP = 50 # deg C
tfinal = 600 # seconds
t_step = 1 # seconds

# create control loop
controller1 = PI(Kp=10.0, Ki=0.5) 
controller1.send(None)

with TCLab() as lab:
    h = Historian(lab.sources)
    p = Plotter(h, tfinal)
    for t in clock(tfinal, t_step):
        PV = lab.T1                                     # measure the the process variable
        MV = lab.U1                                     # get manipulated variable
        MV = controller1.send([t_step, SP, PV, MV])     # PI control to determine the MV
        lab.Q1(MV)                                      # set the heater power
        p.update()                                      # log data

## Exercise 2. Hardware testing the PI controller

* Copy and paste the above code into the cell below. Connect the code to the tclab hardware by changing 'connected' to 'True'.  Adjust the experiment horizon to 1200 seconds to provide plenty of time for testing.

* Test your controller.  Does the performance match the simulation?

* After the controller has achieved the setpoint, introduce a disturbance. An example of a disturbance would be to increase air flow around the device, or to touch the heater with something thermally conductive (be careful, don't use your finger. 50 deg C is hot enough to burn your skin.) 

* Add a text cell below, and comment on your results. Do you see any shortcomings in this control implementation?

In [ ]:
# Add your solution here

Write your comments in this cell.



## Exercise 3. Improved PI Control

Now implmement the most [practical version of the PI controller](../notebooks/03.07-Integral-Windup-and-Bumpless-Transfer.ipynb). You may optionally use the trick above to simulate the system to debug and then try it on your hardware. 

Test your controller with the TCLab hardware. **Be sure to wait until your TCLab cools to room temperature before starting.**

In [ ]:
def PI_Bumpless(Kp=1, Ki=0, MV_bar=0):
    MV = MV_bar
    PV_prev = None
    MV_min = 0
    MV_max = 100
    while True:
        t_step, SP, PV, MV = yield MV
        # Add your solution here
        PV_prev = PV

# Add your solution here

## Excercise 4. Chocolate Tempering Revisited 

In [ ]:
# Add your solution here